# Replication study: Anyone for Tennis (Betting)? von Forrest,D und McHale,I 2007

## Part 1: Descriptive analysis
My goal is to get the same results as in the original study and compare them to new  Data from 2002 to 2020. I use the same data from www.http://www.tennis-data.co.uk/alldata.php. 


In [4]:
import pandas as pd
import numpy as np
import glob
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [5]:
tennis_data = glob.glob("200[2-5].xls")

df1_list = []

for filename in tennis_data:
    df1_list.append(pd.read_excel(filename))

df1 = pd.concat(df1_list).reset_index(drop=True)

tennis_data = glob.glob("20*.xls*")

df2_list = []

for filename in tennis_data:
    df2_list.append(pd.read_excel(filename))

df2 = pd.concat(df2_list).reset_index(drop=True)

In [6]:
Original=df1.copy()
Replication=df2.copy()

In [7]:
Replication.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49534 entries, 0 to 49533
Data columns (total 54 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   ATP         49534 non-null  int64         
 1   Location    49534 non-null  object        
 2   Tournament  49534 non-null  object        
 3   Date        49534 non-null  datetime64[ns]
 4   Series      49534 non-null  object        
 5   Court       49534 non-null  object        
 6   Surface     49534 non-null  object        
 7   Round       49534 non-null  object        
 8   Best of     49534 non-null  int64         
 9   Winner      49534 non-null  object        
 10  Loser       49534 non-null  object        
 11  WRank       49514 non-null  float64       
 12  LRank       49438 non-null  object        
 13  W1          49265 non-null  float64       
 14  L1          49267 non-null  float64       
 15  W2          48780 non-null  object        
 16  L2          48781 non-

### Adjustment for the original study
The autor describes 

In [8]:
# Data only between june 2002 till august 2005
Original = Original.loc[(Original["Date"] >= "2002-06-24") & (Original["Date"] <= "2005-08-27"), :]

In [9]:
# Drop data without available Bet365 odds
Original = Original.dropna(subset=["B365W", "B365L"])
Replication = Replication.dropna(subset=["B365W", "B365L"])

In [10]:
# typing error in the database: correction from 2..3 to 2.3
Replication.loc[32368,"EXW"] =2.3

In [11]:
# change the datatype from object to float
Original["LRank"] = pd.to_numeric(Original.LRank)
Replication["LRank"] = pd.to_numeric(Replication.LRank)
Replication["EXW"] = pd.to_numeric(Replication["EXW"])


In [12]:
# drop rows, where one player could not start the match or had to retire during a match
Original = Original.loc[Original["Comment"] == "Completed", :]
Replication = Replication.loc[(Replication["Comment"] == "Completed") |
                              (Replication["Comment"] == 'Disqualified'), :]
print(
    f"Number of Matches in the original study {Original.shape[0]} vs. {Replication.shape[0]} matches in the replication study")

Number of Matches in the original study 8234 vs. 45031 matches in the replication study


## Deskriptiv analysis
- Odds intervall
- Split into different seasons
- Difference in odds after Woodland and Woodland (1999) 

### Odds intervall (Forrest & McHale, 2007, p. 757, table 1)

In [13]:
# all possible bets into one table
W_Original = pd.DataFrame()
L_Original = pd.DataFrame()

W_Original[["B365W", "Date", "Series"]] = Original[["B365W", "Date", "Series"]]
W_Original["p"] = 1/W_Original["B365W"]
W_Original["return"] = W_Original["B365W"]-1
L_Original[["B365L", "Date", "Series"]] = Original[["B365L", "Date", "Series"]]
L_Original["p"] = 1/L_Original["B365L"]
L_Original["return"] = -1

All_Original = pd.concat([W_Original, L_Original],).sort_values(by="Date").reset_index(drop=True)

In [14]:
# creat odds-intervall in pivot table
All_Original.pivot_table(index=pd.cut(All_Original["p"], np.linspace(
    0, 1, 11), right=False), values="return", aggfunc=["mean", "count", "std"],margins=True)

,mean,count,std
,return,return,return
p,,,
"[0.0, 0.1)",-1.000000,56,0.000000
"[0.1, 0.2)",-0.326113,831,2.028152
"[0.2, 0.3)",-0.262102,1670,1.562675
"[0.3, 0.4)",-0.167762,2062,1.301749
"[0.4, 0.5)",-0.064152,2512,1.124362
"[0.5, 0.6)",-0.071878,2599,0.910989
"[0.6, 0.7)",-0.060572,2668,0.749821
"[0.7, 0.8)",-0.022098,1848,0.601742


- worst mean return (no winners) at the smallest implied proboilities intervall 
- best mean return for betting the biggest favorites
- indication of linearity

In [15]:
# all possible bets into one table
W_Replication = pd.DataFrame()
L_Replication = pd.DataFrame()

W_Replication[["B365W", "Date", "Series"]] = Replication[["B365W", "Date", "Series"]]
W_Replication["p"] = 1/W_Replication["B365W"]
W_Replication["return"] = W_Replication["B365W"]-1
L_Replication[["B365L", "Date", "Series"]] = Replication[["B365L", "Date", "Series"]]
L_Replication["p"] = 1/L_Replication["B365L"]
L_Replication["return"] = -1

All_Replication = pd.concat([W_Replication, L_Replication],).sort_values(by="Date").reset_index(drop=True)

In [16]:
# creat odds-intervall in pivot table
All_Replication.pivot_table(index=pd.cut(All_Replication["p"], np.linspace(
    0, 1, 11), right=False), values="return", aggfunc=["mean", "count", "std"],margins=True)

,mean,count,std
,return,return,return
p,,,
"[0.0, 0.1)",-0.458900,1837,2.870303
"[0.1, 0.2)",-0.274799,5464,2.113373
"[0.2, 0.3)",-0.168300,10306,1.639237
"[0.3, 0.4)",-0.105399,10576,1.331482
"[0.4, 0.5)",-0.060859,12102,1.123391
"[0.5, 0.6)",-0.073420,10210,0.922416
"[0.6, 0.7)",-0.071721,14321,0.757782
"[0.7, 0.8)",-0.036120,10527,0.606393


### Split into different seasons  (Forrest & McHale, 2007, p. 760, table 2)

In [17]:
season02_03 = All_Original.loc[All_Original["Date"] <= "2003-12-31"]

In [18]:
# function to display two tables side by side
# source: https://stackoverflow.com/questions/38783027/jupyter-notebook-display-two-pandas-tables-side-by-side
from IPython.display import display_html
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

In [19]:
season04_05 = All_Original.loc[All_Original["Date"] > "2003-12-31"]

In [20]:
#comparison of seasons 2002 + 2003 (left table) against the seasons 2004 + 2005 (right table)
season02_03=season02_03.pivot_table(index=pd.cut(season02_03["p"], np.linspace(
    0, 1, 11), right=False), values="return", aggfunc=["mean", "count", "std"])
season04_05=season04_05.pivot_table(index=pd.cut(season04_05["p"], np.linspace(
    0, 1, 11), right=False), values="return", aggfunc=["mean", "count", "std"])
display_side_by_side(season02_03,season04_05)

,mean,count,std
,return,return,return
p,,,
"[0.0, 0.1)",-1.000000,4,0.000000
"[0.1, 0.2)",-0.269968,313,2.127308
"[0.2, 0.3)",-0.305952,712,1.516842
"[0.3, 0.4)",-0.144794,948,1.314260
"[0.4, 0.5)",-0.089073,1143,1.122127
"[0.5, 0.6)",-0.068143,1272,0.905440
"[0.6, 0.7)",-0.050943,1167,0.744272
"[0.7, 0.8)",-0.024616,834,0.605185


- very similar mean return
-> the tennis betting market stayed the same over the short time period 

### Grand Slams  (Forrest & McHale, 2007, p. 764, table 4)

In [21]:
GS_Original = All_Original[All_Original["Series"] == "Grand Slam"].copy()
GS_Replication = All_Replication[All_Replication["Series"] == "Grand Slam"].copy()

In [22]:
GS_Original.pivot_table(index=pd.cut(GS_Original["p"], np.linspace(
    0, 1, 11), right=False), values="return", aggfunc=["count","mean", "std"], margins=True)

,count,mean,std
,return,return,return
p,,,
"[0.0, 0.1)",36,-1.000000,0.000000
"[0.1, 0.2)",297,-0.346801,2.059322
"[0.2, 0.3)",365,-0.409674,1.428883
"[0.3, 0.4)",327,-0.070459,1.350127
"[0.4, 0.5)",348,-0.081121,1.125070
"[0.5, 0.6)",335,-0.065555,0.909317
"[0.6, 0.7)",364,-0.066426,0.749257
"[0.7, 0.8)",336,-0.024640,0.600908


In [23]:
GS_Replication.pivot_table(index=pd.cut(GS_Replication["p"], np.linspace(
    0, 1, 11), right=False), values="return", aggfunc=["count","mean", "std"], margins=True)

,count,mean,std
,return,return,return
p,,,
"[0.0, 0.1)",950,-0.568421,2.751961
"[0.1, 0.2)",1785,-0.357423,2.035829
"[0.2, 0.3)",2069,-0.256269,1.576241
"[0.3, 0.4)",1514,-0.105588,1.334939
"[0.4, 0.5)",1653,-0.129964,1.109593
"[0.5, 0.6)",1252,-0.064809,0.922717
"[0.6, 0.7)",1962,-0.021243,0.745222
"[0.7, 0.8)",1624,-0.029310,0.598573


In [31]:
Original.to_csv("Original.csv",index=False)
Replication.to_csv("Replication.csv",index=False)